## Create embeddings for KJV passages from both Train & Test & Create Test Dataframe for Gemma where each KJV-BBE pair in Test has the most similar KJV-BBE pair from Train (based on KJV cosine similarity) to serve as example for in Context Learning 

In [1]:
pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 2.8 MB/s eta 0:00:00ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


### Import packages

In [13]:
import numpy as np
import pandas as pd
import random
import torch
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
pd.set_option('display.max_colwidth', None) 
pd.set_option('display.max_rows', None) 

In [4]:
train_df = pd.read_csv('train_data.csv')

test_df = pd.read_csv('test_data.csv')

### Instantiate the Embedding Model

In [8]:
model = SentenceTransformer('sentence-transformers/all-distilroberta-v1')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Compute embeddings for KJV for both train & test

In [9]:
# Function to compute embeddings for a column in the dataframe
def compute_embeddings(dataframe, column_name):
    sentences = dataframe[column_name].tolist()
    embeddings = model.encode(sentences)
    return embeddings

In [10]:
# Compute embeddings for the 'KJV' column in both dataframes
train_df['KJV_embedding'] = list(compute_embeddings(train_df, 'KJV'))
test_df['KJV_embedding'] = list(compute_embeddings(test_df, 'KJV'))

### For each row in test_df, add in the KJV-BBE pair that has the highest cosine similarity (based on KJV embeddings)

In [14]:
# Convert embeddings from lists to numpy arrays for cosine similarity computation
train_embeddings = np.vstack(train_df['KJV_embedding'].apply(np.array))
test_embeddings = np.vstack(test_df['KJV_embedding'].apply(np.array))

# Compute cosine similarities between all pairs of train and test embeddings
cos_similarities = cosine_similarity(test_embeddings, train_embeddings)

# Find the index of the most similar train embedding for each test row
max_sim_indices = np.argmax(cos_similarities, axis=1)
max_sim_scores = np.max(cos_similarities, axis=1)

# Extract the corresponding KJV, BBE, and embeddings from train_df
train_KJV_nearest = train_df.iloc[max_sim_indices]['KJV'].values
train_BBE_nearest = train_df.iloc[max_sim_indices]['BBE'].values
train_KJV_embedding_nearest = train_df.iloc[max_sim_indices]['KJV_embedding'].values

In [15]:
test_with_nearest_neigh_from_train_df = test_df.copy()
test_with_nearest_neigh_from_train_df['train_KJV'] = train_KJV_nearest
test_with_nearest_neigh_from_train_df['train_BBE'] = train_BBE_nearest
test_with_nearest_neigh_from_train_df['train_KJV_embedding'] = list(train_KJV_embedding_nearest)
test_with_nearest_neigh_from_train_df['cosine_similarity'] = max_sim_scores

In [21]:
test_with_nearest_neigh_from_train_df.sample(2)

,chapter,KJV,BBE,KJV_embedding,train_KJV,train_BBE,train_KJV_embedding,cosine_similarity
51,2Samuel12,"And the Lord sent Nathan unto David. And he came unto him, and said unto him, There were two men in one city; the one rich, and the other poor.","And the Lord sent Nathan to David. And Nathan came to him and said, There were two men in the same town: one a man of great wealth, and the other a poor man.","[0.0019456781, -0.0664589, 0.00020515449, -0.043806337, 0.042802762, 0.019549757, 0.007813624, 0.07654441, -0.02678678, -0.016572053, -0.00031892778, 0.084124774, -0.0014815987, 0.017194562, -0.0926432, 0.067365885, 0.010655339, 0.0074987654, -0.062914275, 0.007568671, 0.05054381, -0.049029566, 0.048862416, -0.011695857, -0.061570685, -0.037357718, -0.0073103267, 0.023696257, -0.03342989, -0.093847916, 0.03635113, -0.081188776, -0.0251645, -0.013169154, -0.005613785, -0.07132958, 0.008278797, -0.017125558, 0.08406128, 0.014549608, -0.034983102, 0.02933466, 0.057265177, 0.023257999, -0.003463733, -0.0018349476, 0.007426777, 0.07096132, 0.012567014, 0.03321697, -0.007759191, 0.009104291, -0.04882992, 0.038205262, 0.0014662221, 0.0017616815, 0.0051250993, 0.100668155, -0.011461984, -0.018603362, 0.011653269, 0.0046729273, -0.050066043, 0.079522416, 0.018111954, -0.015419813, -0.0059340945, -0.0007243474, -0.041703288, -0.021494377, 0.037996843, -0.07972471, 0.004527173, 0.0026382771, -0.025485286, 0.0637218, -0.007726328, -0.013545507, 0.017075919, 0.050411146, 0.013232987, 0.007072926, -0.015268463, -0.004849903, 0.0054924088, 0.055944495, -0.03764179, -0.041724313, 0.023159957, -0.025898667, 0.037764765, 0.05737158, -0.04361379, 0.03327848, 0.0132760955, 0.016934529, -0.020655682, 0.14357357, 0.009647435, -0.026397018, ...]","Then came all the tribes of Israel to David unto Hebron, and spake, saying, Behold, we are thy bone and thy flesh. Also in time past, when Saul was king over us, thou wast he that leddest out and broughtest in Israel: and the Lord said to thee, Thou shalt feed my people Israel, and thou shalt be a captain over Israel.","Then all the tribes of Israel came to David in Hebron and said, Truly, we are your bone and your flesh. In the past when Saul was king over us, it was you who went at the head of Israel when they went out or came in: and the Lord said to you, You are to be the keeper of my people Israel and their ruler.","[0.006571864, -0.05975793, 0.019358555, -0.023215389, 0.07000693, 0.016842127, -0.004783842, 0.06595206, 0.006171807, -0.022626774, -0.029991822, -0.0172439, -0.022506101, 0.0029152008, -0.030179556, 0.08140785, 0.013505134, -0.017631097, -0.02806181, -0.034348924, 0.010026969, -0.049590997, 0.021092758, -0.028717257, 0.034602154, -0.021887805, -0.07521154, 0.010695565, -0.04555428, -0.07236196, -0.006743338, -0.067069404, -0.032423977, -0.008195665, 0.022932949, 0.0011658097, -0.005362318, -0.019001441, 0.101935364, 0.015055844, -0.041378096, 5.0163006e-05, 0.003188094, -0.0174386, -0.041428808, 0.010243367, 0.022319952, 0.03525829, 0.023762845, -0.03899895, 0.0008595437, 0.062342744, -0.020758891, -0.014873612, 0.026265444, 0.021908177, 0.006529504, 0.06271395, 0.015592075, -0.0066196905, 0.042794, -0.031554986, -0.016327877, 0.0270553, -0.049949832, -0.002293736, 0.015283969, -0.064813375, -0.025129158, -0.01908601, 0.05872009, -0.039557472, 0.0063412273, -0.013100029, -0.0075966604, 0.04196706, 0.026816398, -0.031114917, -0.018941037, 0.039244346, 0.060640413, 0.015641917, -0.006994928, -0.019287568, 0.041515615, 0.05448225, -0.037175763, -0.056750115, -0.04376233, 0.014525626, 0.01933515, 0.03910303, -0.018820865, 0.028649297, -0.021085927, 0.04291153, -0.011898216, 0.08997161, -0.03335855, -0.03616893, ...]",0.582918
18,Jeremiah4,"If thou wilt return, O Israel, saith the Lord, return unto me: and if thou wilt put away thine abominations out of my sight, then shalt thou not remove. And thou shalt swear, The Lord liveth, in truth, in judgment, and in righteousness; and the nations sha

In [22]:
test_with_nearest_neigh_from_train_df.to_csv('test_with_nearest_neigh_from_train_df.csv', index=False)